In [33]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import transformers
from torch.optim import Adam
from tqdm import tqdm

In [42]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

In [35]:
from utils import FinQA_Dataset

### Declarations

In [36]:
BATCH_SIZE = 4
NUM_EPOCHS = 2
TESTING = True

In [37]:
TOKENIZER = T5Tokenizer.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.0001)


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: eac10b63-36a7-4aae-b1fa-8099daf46649)')' thrown while requesting HEAD https://huggingface.co/t5-base/resolve/main/spiece.model
/Users/ashish1610dhiman/opt/anaconda3/envs/dlt_project/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length

### Read Data and create DataLoader

In [38]:
finqa_train = torch.load('../finqa_dataset/finqa_train.pth')
finqa_valid = torch.load('../finqa_dataset/finqa_valid.pth')
# torch.save(finqa_test, '../finqa_dataset/finqa_test.pth')

In [41]:
finqa_train

{'input_ids': tensor([125,  19,   8,  ...,   0,   0,   0]),
 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]),
 'labels': tensor([ 5021,   209,    10, 22390,   910,    57,   910,     5,   100,  1527,
             8,   741,    10,     3,  4704,  1713, 30345, 30345, 30345,  5021,
           204,    10, 22390,     3, 26195,    57,     3,  4704,     5,   100,
          1527,     8,   741,    10,     3, 22671,     1,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
        

In [39]:
if TESTING:
    train_loader = DataLoader(Subset(finqa_train, range(1000)), batch_size=BATCH_SIZE)
    val_loader = DataLoader(Subset(finqa_valid, range(1000)), batch_size=BATCH_SIZE)
else:
    train_loader = DataLoader(finqa_train, batch_size=BATCH_SIZE)
    val_loader = DataLoader(finqa_valid, batch_size=BATCH_SIZE)

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

### Train Loop

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

In [ ]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(NUM_EPOCHS):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)
        outputs = MODEL(input_ids=input_ids,attention_mask=attention_mask,
                          labels=labels,decoder_attention_mask=decoder_attention_mask)
        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1
    #Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)
        outputs = MODEL(input_ids=input_ids, attention_mask=attention_mask,
                          labels=labels, decoder_attention_mask=decoder_attention_mask)
        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1
    print(f"{epoch+1}/{NUM_EPOCHS} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")

Training batches:   4%|▎         | 55/1563 [1:54:30<52:19:49, 124.93s/it] 


KeyboardInterrupt: 